In [ ]:
import open3d as o3d
import tensorflow as tf
import numpy as np
import os
from utilities import *
import re
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from pyntcloud import PyntCloud
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from plyfile import PlyData, PlyElement
from scipy.spatial import cKDTree
from tqdm.notebook import tqdm, trange
from multiprocessing import Pool

In [ ]:
initial_path = r'write the path where you put the project'
noisy_modelnet_pcs_path = os.path.join(initial_path, r'noisy_modelnet_pcs')
db_path = os.path.join(initial_path, r'icip2020_perry_quality_repack_bin')
deg_metrics_path = os.path.join('data', 'icip2020_deg_metrics.json')
degraded_pcs_features_path = os.path.join('data', 'icip2020_degraded_pcs_features.csv')
degraded_pcs_features_preds_path = os.path.join('data', 'icip2020_degraded_pcs_features_preds.csv')
block_bits = 6
block_shape = [2**block_bits] * 3
bbox_min = [0,0,0]

## Training on ModelNet dataset

In [ ]:
train_glob = os.path.join(initial_path, r'ModelNet40_200_pc512_oct3_4k/**/*.ply')
files = get_files(train_glob)
assert len(files) > 0
original_modelnet_pcs = []
for path in tqdm(files):
    pc = modelnet_pc(path)
    pc.load_points(path)
    pc.load_tree()
    pc.compute_normals()
    original_modelnet_pcs.append(pc)

In [ ]:
std_list=[]
for k in range(1):
    for i in range(5):
        for j in range(5):
            std_list.append(i+((0.2)*(j+1)))

In [ ]:
noisy_modelnet_pcs = []
d2_list=[]
for pc in tqdm(original_modelnet_pcs) :
    for std in std_list:
        new_pc = make_noisy_version(pc , std)
        new_pc.load_tree()
        new_pc.load_dists_ngbs()
        new_pc.compute_normals()
        new_pc.compute_features()
        d2 = new_pc.features['min_d2_psnr']
        d2_list.append(d2)
        noisy_modelnet_pcs.append(pc.content + '__' + pc.pc_name+ '__' + str(std) + '__' + str(d2))
        new_pc.write_to_disk(noisy_modelnet_pcs_path, d2)

In [ ]:
textfile = open(os.path.join(initial_path,"info_list_noisy_modelnet_pcs.txt"), "w")
for element in noisy_modelnet_pcs :
    textfile.write(element + "\n")
textfile.close()

In [ ]:
noisy_modelnet_pcs = []
L = open(os.path.join(initial_path,"info_list_noisy_modelnet_pcs.txt"), "r").read().splitlines();
for line in L:
  noisy_modelnet_pcs.append(line)

In [ ]:
def shuffle_and_split(number, train_ratio):
    randomlist = np.arange(number)
    np.random.shuffle(randomlist)
    train_randomlist = randomlist[0:round(number*(1-train_ratio))]
    set_randomlist = set(randomlist)
    set_train_randomlist = set(train_randomlist) 
    validation_randomlist = set_randomlist-set_train_randomlist
    train_names = np.array(list(set_train_randomlist))
    np.random.shuffle(train_names)
    validation_names = np.array(list(validation_randomlist))
    np.random.shuffle(validation_names)
    return train_names, validation_names

In [ ]:
def push_example (num): 
    names_list = noisy_modelnet_pcs
    dataset_path = noisy_modelnet_pcs_path
    x1_path = names_list[num] 
    x1 = PyntCloud.from_file(os.path.join(dataset_path, x1_path))
    x1_points = x1.points[['x','y','z']].to_numpy()
    x1 = o3d.geometry.PointCloud()
    x1.points = o3d.utility.Vector3dVector(x1_points)
    R1 = x1.get_rotation_matrix_from_xyz((0, 0, np.random.uniform(0, 2) * np.pi))
    x1.rotate(R1, center = (32,32,32))
    x1.scale(np.random.uniform(0.8,1), center = x1.get_center())
    x1 = np.clip(np.asarray(x1.points), 0, 63)#.to_numpy()
    zeros1 = np.zeros(block_shape, dtype=np.float32)
    x1 = pts_to_vx(x1, block_shape, zeros1)
    x1 = x1.reshape([64,64,64,1])
    for pc in original_modelnet_pcs: 
        if names_list[num].split('__')[2] == pc.pc_name:
            x2_points = pc.points[['x','y','z']].to_numpy()
            x2 = o3d.geometry.PointCloud()
            x2.points = o3d.utility.Vector3dVector(x2_points)
            R2 = x2.get_rotation_matrix_from_xyz((0, 0, np.random.uniform(0, 2) * np.pi))
            x2.rotate(R2, center = (32,32,32))
            x2.scale(np.random.uniform(0.8, 1), center = x2.get_center())
            x2 = np.clip(np.asarray(x2.points), 0, 63)
            break
    zeros2 = np.zeros(block_shape, dtype=np.float32)
    x2 = pts_to_vx(x2, block_shape, zeros2)
    x2 = x2.reshape([64,64,64,1])
    d2 = float(names_list[num].split('__')[-1][:-4])
    d2 = (d2-75)/75
    return x1, x2, d2


def tf_push_example (num):
    return tf.py_function(push_example,[num], [tf.float32, tf.float32, tf.float64])
def divide_example (x1, x2, d2):
    return ((x1, x2), d2)

In [ ]:
train_index, val_index = shuffle_and_split(len(noisy_modelnet_pcs), 0.1)
train_modelnet_dataset = tf.data.Dataset.from_tensor_slices(train_index)#.take(2)
train_modelnet_dataset = train_modelnet_dataset.map(tf_push_example, num_parallel_calls = 64)
train_modelnet_dataset = train_modelnet_dataset.map(divide_example, num_parallel_calls = 64)
train_modelnet_dataset = train_modelnet_dataset.batch(64)
train_modelnet_dataset = train_modelnet_dataset.prefetch(1)
val_modelnet_dataset = tf.data.Dataset.from_tensor_slices(val_index)
val_modelnet_dataset = val_modelnet_dataset.map(tf_push_example, num_parallel_calls = 64)#.take(2)
val_modelnet_dataset = val_modelnet_dataset.map(divide_example, num_parallel_calls = 64)
val_modelnet_dataset = val_modelnet_dataset.batch(64)
val_modelnet_dataset = val_modelnet_dataset.prefetch(1)

In [ ]:
filters=32
block_shape_modified=(64,64,64,1)
params = {'strides': (2, 2, 2), 'padding': 'same', 'use_bias': True}
Embedding = tf.keras.Sequential()
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_0', filters=32, kernel_size=(5, 5, 5), **params, input_shape=block_shape_modified))
Embedding.add(tf.keras.layers.ReLU())
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_1', filters=32, kernel_size=(5, 5, 5), **params))
Embedding.add(tf.keras.layers.ReLU())
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_2', filters=32, kernel_size=(5, 5, 5), **params))
Embedding.add(tf.keras.layers.ReLU())
Embedding.add(tf.keras.layers.Conv3D(name='conv3d_3', filters=8, kernel_size=(1, 1, 1), activation= tf.keras.activations.relu,strides=(1,1,1)))
Embedding.add(tf.keras.layers.Flatten(name='flatten'))
Embedding.add(tf.keras.layers.Dense(32,activation = 'ReLU', name='dense_0'))
Embedding.add(tf.keras.layers.Dense(1,activation = 'ReLU', name='dense_1'))

In [ ]:
right_input = tf.keras.Input((64,64,64,1))
left_input = tf.keras.Input((64,64,64,1))
right_y = Embedding(right_input)
left_y = Embedding(left_input)
out = tf.keras.layers.Subtract()([right_y, left_y]) 
Siamese = tf.keras.Model(inputs = [right_input, left_input ], outputs = [out], name="Siamese")

In [ ]:
checkpoint_filepath = os.path.join(initial_path, r'chekpoints/siamese_modelnet_gaussnoise')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True,
    )
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3)
Siamese.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-02), loss=tf.keras.losses.MeanSquaredError()) 

In [ ]:
history=Siamese.fit(train_modelnet_dataset, epochs=100, callbacks=[callback,reduce_lr,checkpoint_callback], initial_epoch=0 ,validation_data=val_modelnet_dataset)

In [ ]:
Embedding_filepath = os.path.join(initial_path, r'chekpoints/Embedding')
Embedding.save_weights(Embedding_filepath)

## Fine tuning and testing on ICIP 2020 dataset

In [ ]:
df = pd.read_csv(os.path.join(db_path, 'dataset.csv'))
pc_names = df['pc_name'].unique()
df = df.set_index(['pc_name', 'codec_id', 'codec_rate']).sort_index()

In [ ]:
icip_pcs = []
for idx, data in tqdm(df.iterrows()):
    icip_pcs.append(icip_pc(idx[0],idx[1],idx[2],data['geometry_bits'],data['mos'],data['mos_ci'],data['relative_path'],data['radius']))       
for pc in tqdm(icip_pcs):
    pc.load_points()
    pc.connect_with_ref(icip_pcs)
    pc.partition()
    pc.load_tree()
for pc in tqdm(icip_pcs):
    pc.load_dists_ngbs()
    pc.compute_features()
    pc.find_shared_blocks()

In [ ]:
icip_partitions = {}
for name in pc_names:
    icip_block_names = []
    test_names = []
    for pc in icip_pcs:
        if pc.pc_name != name :
            for block in list(pc.blocks_meta.keys()):
                icip_block_names.append([pc.id, block])
        if pc.pc_name == name :
            for block in list(pc.blocks_meta.keys()):
                test_names.append([pc.id, block])
    icip_partitions[name] = {'train' : icip_block_names, 'test' : test_names}

In [ ]:
def shuffle_and_split(number, train_ratio):
    randomlist = np.arange(number)
    np.random.shuffle(randomlist)
    train_randomlist = randomlist[0:round(number*(1-train_ratio))]
    set_randomlist = set(randomlist)
    set_train_randomlist = set(train_randomlist) 
    validation_randomlist = set_randomlist-set_train_randomlist
    train_names = np.array(list(set_train_randomlist))
    np.random.shuffle(train_names)
    validation_names = np.array(list(validation_randomlist))
    np.random.shuffle(validation_names)
    return train_names, validation_names

In [ ]:
def icip_push_sample (num, name, mode): 
    num = num.numpy()
    name = str(name.numpy(),encoding='ascii')
    mode = str(mode.numpy(),encoding='ascii')
    pc_id = icip_partitions[name][mode][num][0]
    block = icip_partitions[name][mode][num][1]
    for pc in icip_pcs:
        if pc.id == pc_id :
            x1 = pc.blocks_meta[block]['block']
            mos = pc.mos/5
    zeros1 = np.zeros(block_shape, dtype=np.float32)
    x1 = pts_to_vx(x1, block_shape, zeros1)
    x1 = x1.reshape([64,64,64,1])
    return x1, mos
def tf_icip_push_sample (num, name, mode):
    return tf.py_function(icip_push_sample, [num, name, mode], [tf.float32, tf.float32])
def icip_divide_sample (x1, d2):
    return (x1, d2)

In [ ]:
for pc in icip_pcs: pc.sum_var = 0
    
for name in tqdm(icip_partitions.keys()):
    
    icip_block_names = icip_partitions[name]
    train_index, val_index = shuffle_and_split(len(icip_block_names), 0.1)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_index)#.take(3)
    name_dataset = tf.data.Dataset.from_tensor_slices(np.asarray([name for i in range(len(train_index))]))
    mode_dataset = tf.data.Dataset.from_tensor_slices(np.asarray(['train' for i in range(len(train_index))]))
    icip_dataset = tf.data.Dataset.zip((train_dataset, name_dataset, mode_dataset))
    icip_dataset = icip_dataset.map(tf_icip_push_sample, num_parallel_calls = 64)
    icip_dataset = icip_dataset.map(icip_divide_sample, num_parallel_calls = 64)
    icip_dataset = icip_dataset.batch(64).prefetch(1)
    
    val_dataset = tf.data.Dataset.from_tensor_slices(val_index)
    icip_dataset_val = tf.data.Dataset.zip((val_dataset, name_dataset, mode_dataset))
    icip_dataset_val = icip_dataset_val.map(tf_icip_push_sample, num_parallel_calls = 64)
    icip_dataset_val = icip_dataset_val.map(icip_divide_sample, num_parallel_calls = 64)
    icip_dataset_val = icip_dataset_val.batch(64).prefetch(1)   

    Embedding.load_weights(Embedding_filepath)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3)
    Embedding.trainable = True
    Embedding.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-04), loss=tf.keras.losses.MeanSquaredError()) 
    history = Embedding.fit(icip_dataset, epochs=100, callbacks=[callback,reduce_lr], initial_epoch=0 ,validation_data=icip_dataset_val)
    
    Embedding.trainable = False
    test_names = icip_partitions[name]['test']
    name_dataset = tf.data.Dataset.from_tensor_slices(np.asarray([name for i in range(len(test_names))]))
    mode_dataset = tf.data.Dataset.from_tensor_slices(np.asarray(['test' for i in range(len(test_names))]))
    test_dataset = tf.data.Dataset.from_tensor_slices(np.arange(len(test_names)))#.take(3)
    icip_dataset_test = tf.data.Dataset.zip((test_dataset, name_dataset, mode_dataset))
    icip_dataset_test = icip_dataset_test.map(tf_icip_push_sample, num_parallel_calls = 128)
    icip_dataset_test = icip_dataset_test.map(icip_divide_sample, num_parallel_calls = 128)
    icip_dataset_test = icip_dataset_test.batch(128).prefetch(1)
    predictions = Embedding.predict(icip_dataset_test)
    
    for i, elem in enumerate(test_names):
        for pc in icip_pcs :
            if pc.id == elem[0] :
                pc.sum_var = pc.sum_var + predictions[i]

In [ ]:
mos_list = np.reshape(np.asarray([pc.mos for pc in icip_pcs]), -1)
preidctions_list = np.reshape(np.asarray([pc.sum_var/pc.num_blocks for pc in icip_pcs]), -1)

In [ ]:
plcc=scipy.stats.pearsonr(mos_list, preidctions_list)
srocc=scipy.stats.spearmanr(mos_list, preidctions_list)

In [ ]:
print(plcc)
print(srocc)